<a href="https://colab.research.google.com/github/lipeng2021/-python1/blob/main/5_10Batch%E2%80%94Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import torch
import torch.nn as nn
from IPython import display
import torchvision
import torchvision.transforms as transforms
import time
FILENAME = '/home/lp'
device = torch.device('cuda'if torch.cuda.is_available()else 'cpu')

In [59]:
def batch_norm(is_training,X,gamma,beta,moving_mean,moving_var,eps,momentum):
  if not is_training:
    X_hat = (X-moving_mean)/torch.sqrt(moving_var**2+eps)
  else:
    assert len(X.shape) in (2,4)
    if len(X.shape) ==2:
      mean = X.mean(dim=0)
      var = ((X-mean)**2).mean(dim=0)
    else:
      mean = X.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
      var = ((X-mean)**2).mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
    X_hat = (X-mean)/torch.sqrt(var**2+eps)
    moving_mean = momentum  *moving_mean +(1.0-momentum)*mean
    moving_var = momentum  *moving_var +(1.0-momentum)*var
  Y = gamma*X_hat+beta
  return Y,moving_mean,moving_var

In [60]:
class BatchNorm(nn.Module):
  def __init__(self,num_features,num_dims):
    super(BatchNorm,self).__init__()
    if num_dims==2:
      shape = (1,num_features)
    else:
      shape = (1,num_features,1,1)
    self.gamma = nn.Parameter(torch.ones(shape))
    self.beta = nn.Parameter(torch.zeros(shape))
    self.moving_mean = torch.zeros(shape)
    self.moving_var = torch.zeros(shape)
  def forward(self,x):
    if self.moving_mean.device !=x.device:
      self.moving_mean = self.moving_mean.to(x.device)
      self.moving_var = self.moving_var.to(x.device)
    Y,self.moving_mean,self.moving_var = batch_norm(self.training,x,self.gamma,self.beta,self.moving_mean,self.moving_var,eps=1e-5,momentum=0.9)
    return Y

In [61]:
class FlattenLayer(nn.Module):
  def __init__(self):
    super(FlattenLayer,self).__init__()
  def forward(self,x):
    return x.view(x.shape[0],-1)

In [61]:
net = nn.Sequential(
    nn.Conv2d(1,6,kernel_size=5),
    BatchNorm(6,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    nn.Conv2d(6,16,kernel_size=5),
    BatchNorm(16,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(kernel_size=2,stride=2),

    FlattenLayer(),

    nn.Linear(16*4*4,120),
    BatchNorm(120,num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120,84),
    BatchNorm(84,num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84,10)
)

In [62]:
net1 = nn.Sequential(
    nn.Conv2d(1,6,kernel_size=5),
    nn.BatchNorm2d(6,eps=1e-5,momentum=0.9),
    nn.Sigmoid(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    nn.Conv2d(6,16,kernel_size=5),
    nn.BatchNorm2d(16,eps=1e-5,momentum=0.9),
    nn.Sigmoid(),
    nn.MaxPool2d(kernel_size=2,stride=2),

    FlattenLayer(),

    nn.Linear(16*4*4,120),
    nn.BatchNorm1d(120,eps=1e-5,momentum=0.9),
    nn.Sigmoid(),
    nn.Linear(120,84),
    nn.BatchNorm1d(84,eps=1e-5,momentum=0.9),
    nn.Sigmoid(),
    nn.Linear(84,10)
)

In [63]:
def load_data_fashion_mnist(batch_size,resize=None,root=FILENAME):
  trans = []
  if resize:
    trans.append(transforms.Resize(size=resize))
  trans.append(transforms.ToTensor())
  transform = transforms.Compose(trans)
  mnist_train = torchvision.datasets.FashionMNIST(root=root,train=True,transform=transform,download=True)
  mnist_test = torchvision.datasets.FashionMNIST(root=root,train=False,transform=transform,download=True)
  train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=0)
  test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=0)
  return train_iter,test_iter
batch_size=64
train_iter,test_iter = load_data_fashion_mnist(batch_size)

In [64]:
def evaculate_accuracy(data_iter,net):
  acc_sum = 0.0
  n = 0
  with torch.no_grad():
    for X,y in data_iter:
      if isinstance(net,nn.Module):
        acc_sum += (net(X.to(device)).argmax(dim=1)==y.to(device)).float().sum().cpu().item()
      else:
        if ('is_training'in net.__code__varnames):
          acc_sum += (net(X,is_training=False).argmax(dim=1)==y).float().sum().cpu().item()
        else:
          acc_sum += (net(X).argmax(dim=1)==y).float().sum().cpu().item()
      n += y.shape[0]
    return acc_sum/n

In [65]:
def train_ch5(net,train_iter,test_iter,batch_size,optimizer,num_epochs):
  loss = nn.CrossEntropyLoss()
  net = net.to(device)
  print('training on:',device)
  for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc_sum = 0.0
    n = 0
    batch_cout = 0
    start = time.time()
    for X,y in train_iter:
      X = X.to(device)
      y = y.to(device)
      y_hat = net(X)
      l = loss(y_hat,y)
      optimizer.zero_grad()
      l.backward()
      optimizer.step()
      train_loss += l.cpu().item()
      train_acc_sum += (net(X).argmax(dim=1)==y).float().sum().cpu().item()
      n += y.shape[0]
      batch_cout += 1
    test_acc_sum = evaculate_accuracy(test_iter,net)
    print('epoch:%d,loss:%.4f,train_acc:%.3f,test_acc:%.3f,time:%.1f'%(epoch+1,train_loss/batch_cout,train_acc_sum/n,test_acc_sum,time.time()-start))

In [67]:
lr = 0.001
num_epochs =5
optimizer = torch.optim.Adam(net.parameters(),lr=lr)
optimizer = torch.optim.Adam(net1.parameters(),lr=lr)
train_ch5(net1,train_iter,test_iter,batch_size,optimizer,num_epochs)
train_ch5(net,train_iter,test_iter,batch_size,optimizer,num_epochs)


training on: cuda
epoch:1,loss:0.6544,train_acc:0.833,test_acc:0.853,time:9.0
epoch:2,loss:0.3639,train_acc:0.880,test_acc:0.876,time:9.0
epoch:3,loss:0.3245,train_acc:0.891,test_acc:0.876,time:9.0
epoch:4,loss:0.3008,train_acc:0.898,test_acc:0.881,time:9.0
epoch:5,loss:0.2821,train_acc:0.903,test_acc:0.885,time:9.0
training on: cuda


TypeError: ignored